## **Урок 9. Языковое моделирование**
###Задание

Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор текстов

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = '12_chairs.txt'

In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 131708 characters


In [4]:
print(text[:500])

Двенадцать стульев. Полная версия романа.
Евгений Петрович Петров

Глава I

Безенчук и нимфы

В уездном городе N было так много парикмахерских заведений и бюро похоронных процессий, что, казалось, жители города рождаются лишь затем, чтобы побриться, остричься, освежить голову вежеталем и сразу же умереть. А на самом деле в уездном городе N люди рождались, брились и умирали довольно редко. Жизнь города была тишайшей. Весенние вечера были упоительны, грязь под луною сверкала, как антрацит, 


In [5]:
text = text + text

In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

103 unique characters


In [7]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
text_as_int, text, len(text_as_int), len(text)

(array([42, 68, 71, ..., 80, 37,  9]),
 'Двенадцать стульев. Полная версия романа.\r\nЕвгений Петрович Петров\r\n\r\nГлава I\r\n\r\nБезенчук и нимфы\r\n\r\nВ уездном городе N было так много парикмахерских заведений и бюро похоронных процессий, что, казалось, жители города рождаются лишь затем, чтобы побриться, остричься, освежить голову вежеталем и сразу же умереть. А на самом деле в уездном городе N люди рождались, брились и умирали довольно редко. Жизнь города была тишайшей. Весенние вечера были упоительны, грязь под луною сверкала, как антрацит, и вся молодежь города до такой степени была влюблена в секретаршу месткома коммунальников, что это просто мешало ей собирать членские взносы.\r\n\r\nВопросы любви и смерти не волновали Ипполита Матвеевича Воробьянинова, хотя этими вопросами, по роду своей службы, он ведал с 9 утра до 5 вечера ежедневно, с получасовым перерывом для завтрака.\r\n\r\nПо утрам, выпив из причудливого (морозного с жилкой) стакана свою порцию горячего молока, подан

In [9]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

Д
в
е
н
а


In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Двенадцать стульев. Полная версия романа.\r\nЕвгений Петрович Петров\r\n\r\nГлава I\r\n\r\nБезенчук и нимфы\r\n\r\n'
'В уездном городе N было так много парикмахерских заведений и бюро похоронных процессий, что, казалось'
', жители города рождаются лишь затем, чтобы побриться, остричься, освежить голову вежеталем и сразу ж'
'е умереть. А на самом деле в уездном городе N люди рождались, брились и умирали довольно редко. Жизнь'
' города была тишайшей. Весенние вечера были упоительны, грязь под луною сверкала, как антрацит, и вся'


In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Двенадцать стульев. Полная версия романа.\r\nЕвгений Петрович Петров\r\n\r\nГлава I\r\n\r\nБезенчук и нимфы\r\n\r'
Target data: 'венадцать стульев. Полная версия романа.\r\nЕвгений Петрович Петров\r\n\r\nГлава I\r\n\r\nБезенчук и нимфы\r\n\r\n'


In [13]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [14]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [15]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ]
)

In [16]:
class RNNgenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, batch_size):
        super(RNNgenerator, self).__init__()
        
        self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
                                 
        self.gru1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
        self.gru2 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
                           
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        emb_x = self.emb(x)
        x1 = self.gru1(emb_x)
        x = x1
        for _ in range(3):
            x = self.gru2(x)
        #x = self.gru1(x)
        x = (x + x1)/2
        return self.fc(x)

model = RNNgenerator(vocab_size, embedding_dim, BATCH_SIZE)

In [17]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [18]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 103) # (batch_size, sequence_length, vocab_size)


In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (64, None, 256)           26368     
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 lstm_2 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_3 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_4 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 dense_2 (Dense)             (64, None, 103)           105575    
                                                                 
Total params: 30,557,031
Trainable params: 30,557,031


In [21]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 103), dtype=float32, numpy=
array([[ 1.0995098e-05, -2.7572007e-05,  2.5912408e-05, ...,
        -1.1872928e-05, -1.9178464e-05,  8.3100649e-06],
       [ 5.3640542e-05, -8.5514614e-05,  8.6089567e-05, ...,
        -5.7867030e-05, -3.6822748e-05, -1.1099928e-05],
       [ 1.0547470e-04, -1.5010015e-04,  1.5920989e-04, ...,
        -1.4677044e-04, -6.8042144e-05, -6.8695990e-05],
       ...,
       [ 2.6858086e-04,  1.1187614e-04,  5.2564777e-04, ...,
        -1.7071627e-03, -1.3097575e-03, -6.5316050e-04],
       [ 2.9127172e-04,  2.1656271e-04,  3.5674684e-04, ...,
        -1.8302767e-03, -1.2671163e-03, -5.9964741e-04],
       [ 3.1276152e-04,  3.0422065e-04,  2.4644588e-04, ...,
        -1.9683002e-03, -1.2314518e-03, -6.0902094e-04]], dtype=float32)>

In [22]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [23]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'рамора были наклеены приказы по дому №\xa02 Старсобеса. Остап читал их, время от времени энергично спра'

Next Char Predictions: 
 'aОovХж.iие№;ЛэNюОс–Нк8-(Ри№ВМПо—,Ч?ПаМъ5ж-ЛфaЧЖoЩ8vАo5ЩЗПаОu–’—ащ.\r№6тгЩэ99йбыШ?ЖuфТл6—9Жп/\rК\rо’ЕПаС'


In [24]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 103)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.634776


In [25]:
model.compile(optimizer='adam', loss=loss)

In [26]:
!rm -rf ./training_checkpoints

In [27]:
!ls ./training_checkpoints

ls: cannot access './training_checkpoints': No such file or directory


In [28]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*5,
    save_weights_only=True)

In [29]:
EPOCHS = 60
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/60
40/40 [==============================] - 24s 383ms/step - loss: 3.7200
Epoch 2/60
40/40 [==============================] - 16s 383ms/step - loss: 3.3632
Epoch 3/60
40/40 [==============================] - 15s 365ms/step - loss: 3.3640
Epoch 4/60
40/40 [==============================] - 15s 362ms/step - loss: 3.3630
Epoch 5/60
40/40 [==============================] - 15s 372ms/step - loss: 3.3635
Epoch 6/60
40/40 [==============================] - 15s 380ms/step - loss: 3.3624
Epoch 7/60
40/40 [==============================] - 15s 371ms/step - loss: 3.3627
Epoch 8/60
40/40 [==============================] - 15s 366ms/step - loss: 3.3631
Epoch 9/60
40/40 [==============================] - 15s 364ms/step - loss: 3.3615
Epoch 10/60
40/40 [==============================] - 15s 365ms/step - loss: 3.3623
Epoch 11/60
40/40 [==============================] - 16s 390ms/step - loss: 3.3615
Epoch 12/60
40/40 [==============================] - 15s 370ms/step - loss: 3.3627
Epoch 13/60
4

In [30]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_55'

In [31]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [32]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (1, None, 256)            26368     
                                                                 
 lstm_5 (LSTM)               (1, None, 1024)           5246976   
                                                                 
 lstm_6 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_7 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_8 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 dense_3 (Dense)             (1, None, 103)            105575    
                                                                 
Total params: 30,557,031
Trainable params: 30,557,031


In [34]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = t

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [41]:
t = 0.3 # подбираем температуру, в процессе подбора пару раз получилось довольно смешно (например, бритва "Сделал свое дело - и уходи")

In [42]:
text_ = generate_text(model, start_string=u"Вот вам от мертвого осла уши ")
print(text_)

Вот вам от мертвого осла уши тоже не бредит.

– Когда же вы помните, как граждане продать его за городской гимназии. Они называли их в кармана старую бритву «Сделал свое дело – и уходи». В коробки раздраженно сказал:

– Читал.

– Ну вот-с, – угрюмо ответил Ипполит Матвеевич подстригать породы своим делам. Недавно прислала мне предложить им может выстоять.

Остап подал дирижеру руку и спиною налетел на крыльцо. В зеркальце отразился с вывесками и привычка к кругах сорвиголова, радостно заимать Ипполиту двойку, но это
